# Federated Learning with Flower (`flwr`)

[Flower](https://flower.dev/) (`flwr`) is a framework for building federated learning systems. The
design of Flower is based on a few guiding principles:

* **Customizable**: Federated learning systems vary wildly from one use case to
  another. Flower allows for a wide range of different configurations depending
  on the needs of each individual use case.

* **Extendable**: Flower originated from a research project at the University of
  Oxford, so it was built with AI research in mind. Many components can be
  extended and overridden to build new state-of-the-art systems.

* **Framework-agnostic**: Different machine learning frameworks have different
  strengths. Flower can be used with any machine learning framework, for
  example, [PyTorch](https://pytorch.org),
  [TensorFlow](https://tensorflow.org), [Hugging Face Transformers](https://huggingface.co/), [PyTorch Lightning](https://pytorchlightning.ai/), [MXNet](https://mxnet.apache.org/), [scikit-learn](https://scikit-learn.org/), [JAX](https://jax.readthedocs.io/), [TFLite](https://tensorflow.org/lite/), [Pandas](https://pandas.pydata.org/
) for federated analytics, or even raw [NumPy](https://numpy.org/)
  for users who enjoy computing gradients by hand.

* **Understandable**: Flower is written with maintainability in mind. The
  community is encouraged to both read and contribute to the codebase.

## Flower with TensorFlow

To start with, let' try to build a federated learning system using Flower and [TensorFlow](https://www.tensorflow.org/). 

### Install required packages

1. Check out Python versions supporting both TensorFlow and Flower. ([Flower's](https://flower.dev/docs/installation.html#python-version) and [TensorFlow's](https://www.tensorflow.org/install) requirements) 

2. Create a virtual environment using [conda](https://docs.conda.io/en/latest/), [virtualenv](https://virtualenv.pypa.io/en/latest/) or other tools. Let's say you use Anaconda. 

```shell
conda create -n flwr python # you can specify a Python version: "python" => "python=3.10"
```

3. Activate that environemnt.

```shell
conda activate flwr
```

4. Install TensorFlow and Flower

```shell
# Install Tensorflow
# Check out https://www.tensorflow.org/install/pip for latest instruction
# You may need to change codatoolkit and cudnn version depending on your setup and any future change to TensorFlow
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
python3 -m pip install tensorflow
# Verify install:
python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

# Install Flower
pip install flwr
```

And we are good to go!

### Design server and clients

Flower has made this step very easy. 

#### Clients

Create a file `client.py`. And in that file, write

```python
import flwr as fl
import tensorflow as tf
```

to import the required modules. 

After that, let's load the famous [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset to do a simple image classification job. 

```python
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
```

For the model, let's use [MobileNetV2](https://arxiv.org/abs/1801.04381), a convolutional neural network (CNN) architecture that seeks to perform well on mobile devices, with 10 output classes.

```python
model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
```

![mnv2](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-06_at_10.37.14_PM.png)

The Flower server interacts with clients through an interface called `Client`. When the server selects a particular client for training, it sends training instructions over the network. The client receives those instructions and calls one of the `Client` methods to run your code (i.e., to train the neural network we defined earlier).

Flower provides a convenience class called `NumPyClient` which makes it easier to implement the `Client` interface when your workload uses Keras. The `NumPyClient` interface defines three methods which can be implemented in the following way:

```python
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        model.fit(x_train, y_train, epochs=1, batch_size=32, steps_per_epoch=3)
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(x_test, y_test)
        return loss, len(x_test), {"accuracy": float(accuracy)}
```

Now create an instance of our class CifarClient and add one line to actually run this client:

```python
fl.client.start_numpy_client(server_address="[::]:8080", client=CifarClient())
```

The string `"[::]:8080"` tells the client which server to connect to. In our case we can run the server and the client on the same machine, therefore we use `"[::]:8080"`. If we run a truly federated workload with the server and clients running on different machines, all that needs to change is the `server_address` we point the client at.

#### Server

Server setup is even easier as Flower has simplified more for us. Create a file `server.py` and in that file:

```python
import flwr as fl

fl.server.start_server(config=fl.server.ServerConfig(num_rounds=3))
```

### Train the model

1. Start the server:

```shell
python server.py
```

2. In a different terminal sessions, run

```shell
$ python client.py
```

to start a first client and in another terminal session run the same code for a second one.

Voilà! FL has started. You should be able to see in the server terminal:

```shell
INFO flower 2021-02-25 14:15:46,741 | app.py:76 | Flower server running (insecure, 3 rounds)
INFO flower 2021-02-25 14:15:46,742 | server.py:72 | Getting initial parameters
INFO flower 2021-02-25 14:16:01,770 | server.py:74 | Evaluating initial parameters
INFO flower 2021-02-25 14:16:01,770 | server.py:87 | [TIME] FL starting
DEBUG flower 2021-02-25 14:16:12,341 | server.py:165 | fit_round: strategy sampled 2 clients (out of 2)
DEBUG flower 2021-02-25 14:21:17,235 | server.py:177 | fit_round received 2 results and 0 failures
DEBUG flower 2021-02-25 14:21:17,512 | server.py:139 | evaluate: strategy sampled 2 clients
DEBUG flower 2021-02-25 14:21:29,628 | server.py:149 | evaluate received 2 results and 0 failures
DEBUG flower 2021-02-25 14:21:29,696 | server.py:165 | fit_round: strategy sampled 2 clients (out of 2)
DEBUG flower 2021-02-25 14:25:59,917 | server.py:177 | fit_round received 2 results and 0 failures
DEBUG flower 2021-02-25 14:26:00,227 | server.py:139 | evaluate: strategy sampled 2 clients
DEBUG flower 2021-02-25 14:26:11,457 | server.py:149 | evaluate received 2 results and 0 failures
DEBUG flower 2021-02-25 14:26:11,530 | server.py:165 | fit_round: strategy sampled 2 clients (out of 2)
DEBUG flower 2021-02-25 14:30:43,389 | server.py:177 | fit_round received 2 results and 0 failures
DEBUG flower 2021-02-25 14:30:43,630 | server.py:139 | evaluate: strategy sampled 2 clients
DEBUG flower 2021-02-25 14:30:53,384 | server.py:149 | evaluate received 2 results and 0 failures
INFO flower 2021-02-25 14:30:53,384 | server.py:122 | [TIME] FL finished in 891.6143046000007
INFO flower 2021-02-25 14:30:53,385 | app.py:109 | app_fit: losses_distributed [(1, 2.3196680545806885), (2, 2.3202896118164062), (3, 2.1818180084228516)]
INFO flower 2021-02-25 14:30:53,385 | app.py:110 | app_fit: accuracies_distributed []
INFO flower 2021-02-25 14:30:53,385 | app.py:111 | app_fit: losses_centralized []
INFO flower 2021-02-25 14:30:53,385 | app.py:112 | app_fit: accuracies_centralized []
DEBUG flower 2021-02-25 14:30:53,442 | server.py:139 | evaluate: strategy sampled 2 clients
DEBUG flower 2021-02-25 14:31:02,848 | server.py:149 | evaluate received 2 results and 0 failures
INFO flower 2021-02-25 14:31:02,848 | app.py:121 | app_evaluate: federated loss: 2.1818180084228516
INFO flower 2021-02-25 14:31:02,848 | app.py:125 | app_evaluate: results [('ipv4:127.0.0.1:57158', EvaluateRes(loss=2.1818180084228516, num_examples=10000, accuracy=0.0, metrics={'accuracy': 0.21610000729560852})), ('ipv4:127.0.0.1:57160', EvaluateRes(loss=2.1818180084228516, num_examples=10000, accuracy=0.0, metrics={'accuracy': 0.21610000729560852}))]
INFO flower 2021-02-25 14:31:02,848 | app.py:127 | app_evaluate: failures [] flower 2020-07-15 10:07:56,396 | app.py:77 | app_evaluate: failures []
```

## Further reading and experiments

- [Flower documentation](https://flower.dev/docs/)

- [TensorFlow documentation](https://www.tensorflow.org/)

- [MobileNetV2 paper](https://arxiv.org/abs/1801.04381)

- [TensorFlow Federated](https://www.tensorflow.org/federated)

- [FATE](https://fate.fedai.org/)

- [IBM FL](https://ibmfl.mybluemix.net/)